# 统计每个玩家参与的游戏局数情况

In [5]:
import sqlite3
conn=sqlite3.connect(r"C:\sqlite3\chess.db")
cursor=conn.cursor()

In [ ]:
cursor.execute('''
    PRAGMA table_info(players);
''')
columns = [column[1] for column in cursor.fetchall()]
if 'TotalGames' not in columns:
    cursor.execute('ALTER TABLE players ADD COLUMN TotalGames INT DEFAULT 0;')
if 'WonGames' not in columns:
    cursor.execute('ALTER TABLE players ADD COLUMN WonGames INT DEFAULT 0;')
if 'LostGames' not in columns:
    cursor.execute('ALTER TABLE players ADD COLUMN LostGames INT DEFAULT 0;')
if 'DrawnGames' not in columns:
    cursor.execute('ALTER TABLE players ADD COLUMN DrawnGames INT DEFAULT 0;')

cursor.execute('''
    SELECT 
        White AS player, 
        COUNT(*) AS TotalGames,
        SUM(CASE WHEN Result = '1-0' THEN 1 ELSE 0 END) AS WonGames,
        SUM(CASE WHEN Result = '0-1' THEN 1 ELSE 0 END) AS LostGames,
        SUM(CASE WHEN Result = '1/2-1/2' THEN 1 ELSE 0 END) AS DrawnGames
    FROM games
    GROUP BY White
    UNION
    SELECT 
        Black AS player, 
        COUNT(*) AS TotalGames,
        SUM(CASE WHEN Result = '0-1' THEN 1 ELSE 0 END) AS WonGames,
        SUM(CASE WHEN Result = '1-0' THEN 1 ELSE 0 END) AS LostGames,
        SUM(CASE WHEN Result = '1/2-1/2' THEN 1 ELSE 0 END) AS DrawnGames
    FROM games
    GROUP BY Black
''')

player_stats = {}
for row in cursor.fetchall():
    player_stats[row[0]] = {
        'TotalGames': row[1],
        'WonGames': row[2],
        'LostGames': row[3],
        'DrawnGames': row[4]
    }

for player, stats in player_stats.items():
    cursor.execute('''
        UPDATE players
        SET 
            TotalGames = ?, 
            WonGames = ?, 
            LostGames = ?, 
            DrawnGames = ?
        WHERE user = ?
    ''', (stats['TotalGames'], stats['WonGames'], stats['LostGames'], stats['DrawnGames'], player))

conn.commit()
conn.close()

print("Players table has been successfully updated.")

Players table has been successfully updated.
